# Импорт и настройки

Для вывода графиков в ноутбуке

In [2]:
%matplotlib inline

Установка необходимых библиотек

In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

Импорт библиотек

In [2]:
import os
import cv2
import torch
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.2.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)
Setup complete ✅ (12 CPUs, 31.2 GB RAM, 34.4/102.2 GB disk)


Проверка доступности `CUDA` и GPU

In [3]:
if torch.cuda.is_available():    
    print('CUDA is available. Version:', torch.version.cuda)
    print('CUDA device:', torch.cuda.get_device_name(0))
else:
    print('CUDA is not available.')

!nvidia-smi

CUDA is available. Version: 12.1
CUDA device: NVIDIA GeForce RTX 2060
Sat May 25 12:08:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0              27W /  80W |      8MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A 

Константы

In [5]:
EPOCHS = 300    # подобрано опытным путём
IMG_SIZE = 640  # размер изображений датасета (416 для data_folder_1)
BATCH_SIZE = 8  # количество изображений в батче, зависит от размера памяти GPU
LR = 0.001      # начальная скорость обучения (игнорируется при auto оптимизаторе)
# Оптимизатор для тренировки(SGD, Adam, AdamW, NAdam, RAdam, RMSProp, auto),
# влияет на скорость сходимости и стабильность
OPTIMIZER = 'auto'
MODEL_NAME = 'yolov9_d7_e300'  # имя для хранения обученной модели

Пути к папкам и файлам

In [6]:
# Получаем путь к текущей папке /src
# current_dir = os.path.dirname(os.path.abspath(__file__))  # для .py
current_dir = os.getcwd()  # для .ipynb

# Находим папку проекта относительно папки /src
project_path = os.path.abspath(os.path.join(current_dir, os.pardir))
print(f"Путь к папке проекта {project_path}")

# Задаём путь к папке данных относительно папки проекта
# data_folder_0 = os.path.join(project_path, 'theft-detection-97nip2_YOLOv9')
# data_folder_1 = os.path.join(project_path, 'crime-detection-v8i-yolov9')
# data_folder_2 = os.path.join(project_path, 'Crime CCTV Object Detection.v1i.yolov9')
# data_folder_3 = os.path.join(project_path, 'Crime CCTV Object Detection.v2i.yolov9')
# data_folder_4 = os.path.join(project_path, 'Crime CCTV Object Detection.v3i.yolov9')
# data_folder_5 = os.path.join(project_path, 'Crime CCTV Object Detection.v5i.yolov9')
# data_folder_6 = os.path.join(project_path, 'Crime CCTV Object Detection.v6i.yolov9')
data_folder_7 = os.path.join(project_path, 'Crime CCTV Object Detection.v7i.yolov9')
data_folder_8 = os.path.join(project_path, 'Crime CCTV Object Detection.v8i.yolov9')

# Задаём путь к датасету
dataset_file = 'data.yaml'
dataset_path = os.path.join(data_folder_7, dataset_file)
print(f"Путь к датасету {data_folder_7}")

# Задаём путь к текущей моделе
model_path = os.path.join(current_dir, 'runs', 'detect', MODEL_NAME)

# Список  моделей
models = [
    MODEL_NAME,         # текущая модель
    # 'yolov8n_d7_e300',
    # 'yolov9_d7_e300',
    # 'yolov9_d7_e161',
    # 'yolov9_d7_e100',
    # 'yolov9_d6_e100',
    # 'yolov9_d5_e100',
    # 'yolov9_d4_e100',
    # 'yolov9_d3_50_s2',  # дообучение 'yolov9_d3_e50_aug'
    # 'yolov9_d3_e50_aug',
    # 'yolov9_d2_e50',
    # 'yolov9_d1_e50_noaug',
    # 'yolov9_d0_e100'
]

# Создание путей к файлам результатов для каждой модели
results_file = 'results.csv'
results_path = {key: os.path.join(current_dir, 'runs', 'detect', key, results_file) for key in models}
print(f"Путь к файлу результатов {results_path[MODEL_NAME]}")

# Для сохранения результатов
best_file = 'best.pt'
output_path = os.path.join(model_path, 'weights', best_file)
print(f"Путь к файлу модели {output_path}")

# Путь к метрикам на тестовой выборке
inference_path = os.path.join(model_path, 'inference')

# Для вывода тестовых результатов
'''
test_file1 = 'Theft--24-_mp4-40_jpg.rf.564b88b5933f327e08d7f7ebdf8c4cde.jpg'  # data_folder_0
test_file2 = 'Theft--41-_mp4-3_jpg.rf.83208eab8d5ff757539623f62b191571.jpg'
test_files = ['burg10_0118_jpg.rf.0254bd0bb0495327ea538559c0447f46.jpg',  # data_folder_1
              'figV_5_0003_jpg.rf.41914284c4850e713aa12f4e4ba7705e.jpg',
              'ROb16_0032_jpg.rf.1c52eb334bed44102d57ff5a3931b0ae.jpg']
test_files = ['burg10_0118_jpg.rf.35badf5be6e43348abdb871202788cb0.jpg',  # data_folder_2 - 3
              'figV_5_0003_jpg.rf.267c49735a53d1544ed6afac051959cc.jpg',
              'ROb16_0032_jpg.rf.86133a8bd41af7b00e3187e3438f0410.jpg']
test_files = ['burg10_0118_jpg.rf.dafbf0189d498ee855f2e3242b98fc22.jpg',  # data_folder_4 - 5
              'figV_5_0003_jpg.rf.bbf327f337e28d84c2bbd87df740277a.jpg',
              'ROb16_0032_jpg.rf.223122dde647aaf56da9531e5fd4a916.jpg']
'''
test_files = ['burg10_0118_jpg.rf.dafbf0189d498ee855f2e3242b98fc22.jpg',  # data_folder_6 - 7
              'figV_5_0003_jpg.rf.8e76dd927944ee7b71037335f5552fb2.jpg',
              'ROb41_0017_jpg.rf.708ca2b763fe3b421d6b5980b1cf25d7.jpg']

test_images_path = os.path.join(project_path, data_folder_8, 'test', 'images')
test_path3 = []
for i in range(3):
    test_path3.append(os.path.join(test_images_path, test_files[i]))

# Для возобновления обучения
last_file = 'last.pt'
resume_path = os.path.join(model_path, 'weights', last_file)

Путь к папке проекта /media/igor/WORK/ML/Диплом
Путь к датасету /media/igor/WORK/ML/Диплом/Crime CCTV Object Detection.v7i.yolov9
Путь к файлу результатов /media/igor/WORK/ML/Диплом/src/runs/detect/yolov9_d7_e300/results.csv
Путь к файлу модели /media/igor/WORK/ML/Диплом/src/runs/detect/yolov9_d7_e300/weights/best.pt


Загрузка лучшей обученной модели

In [7]:
model_yolov9_best = YOLO(output_path)
print(MODEL_NAME)
model_yolov9_best.info()  # для вывода информации о модели

yolov9_d7_e300
YOLOv9c summary: 618 layers, 25531545 parameters, 0 gradients, 103.7 GFLOPs


(618, 25531545, 0, 103.69152)

Предсказание на видео не из датасета

In [8]:
video_file = 'robbery.gif'  # robbery !
video_path = os.path.join(current_dir, video_file)
results = model_yolov9_best.predict(video_path, conf=0.5, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



: 

Очистка видеокэша `PyTorch`

In [4]:
torch.cuda.empty_cache()